In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from   sklearn.cluster import KMeans
from   sklearn.linear_model import LinearRegression
from   sklearn.linear_model import LogisticRegression

In [108]:
df = pd.DataFrame(columns=['h', 'a', 'h g', 'a g', 'h xg', 'a xg', 'h npxg', 'a npxg', 'h elo', 'a elo'])
form_matches = 20
for filename in ['EPL_2018_w_ELO.csv', 'EPL_2019_w_ELO.csv', 'EPL_2020_w_ELO.csv', 'EPL_2021_w_ELO.csv']:
    temp = pd.read_csv(filename)
    
    for i in temp.index:
        print(i, end='\r')
        home_team = temp.loc[i,'Home Team']
        away_team = temp.loc[i,'Away Team']
        home_g_form, away_g_form  = list(), list()
        home_npxg_form, away_npxg_form = list(), list()
        indexer = df.shape[0]-1
        
        while indexer >= 0 \
                and len(home_g_form)<form_matches and len(away_g_form)<form_matches \
                and len(home_npxg_form)<form_matches and len(away_npxg_form)<form_matches:
            team_names = df.loc[indexer,['h', 'a']].values
            
            if home_team in team_names:
                if len(home_npxg_form) < form_matches:
                    home_npxg_form.append(df.loc[indexer,'h npxg'])
                if len(home_g_form) < form_matches:
                    home_g_form.append(df.loc[indexer,'h g'])
            
            if away_team in team_names:
                if len(away_npxg_form) < form_matches:
                    away_npxg_form.append(df.loc[indexer,'a npxg'])
                if len(away_g_form) < form_matches:
                    away_g_form.append(df.loc[indexer,'a g'])
            indexer -= 1
        
        df = df.append(pd.Series({'h': home_team,
                                  'a': away_team,
                                  'h g': temp.loc[i,'Home Goals'], 
                                  'a g': temp.loc[i,'Away Goals'], 
                                  'h xg': temp.loc[i,'FBRef Home xG'], 
                                  'a xg': temp.loc[i,'FBRef Away xG'], 
                                  'h npxg': temp.loc[i,'FBRef Home npxG'], 
                                  'a npxg': temp.loc[i,'FBRef Away npxG'], 
                                  'h elo': temp.loc[i,'Home ELO'], 
                                  'a elo': temp.loc[i,'Away ELO'],
                                  'h g form': np.mean(home_g_form),
                                  'a g form': np.mean(away_g_form),
                                  'h npxg form': np.mean(home_npxg_form),
                                  'a npxg form': np.mean(away_npxg_form)}), 
                       ignore_index=True)
    print('Done with {}'.format(filename))

C:\Users\Owner\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Owner\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Done with EPL_2018_w_ELO.csv
Done with EPL_2019_w_ELO.csv
Done with EPL_2020_w_ELO.csv
Done with EPL_2021_w_ELO.csv


In [101]:
# x = (df['h elo']-df['a elo']).values.reshape(-1,1)
# y = (df['h g']-df['a g']).values.reshape(-1,1)
# model = LinearRegression().fit(x, y)

# %matplotlib inline
# fig, ax = plt.subplots(figsize=[18,12])
# ax.scatter(x, y, alpha=0.1)
# ax.plot([min(x), max(x)], 
#         [min(x)*model.coef_[0][0]+model.intercept_, max(x)*model.coef_[0][0]+model.intercept_],
#         color='red')
# plt.show()

In [102]:
# x = np.concatenate([df['h elo']-df['a elo'],
#                     df['a elo']-df['h elo']]).reshape(-1,1)
# y = np.concatenate([df['h g'], df['a g']]).reshape(-1,1)
# model = LinearRegression().fit(x, y)

# %matplotlib inline
# fig, ax = plt.subplots(figsize=[18,12])
# ax.scatter(x, y, alpha=0.05)
# ax.plot([min(x), max(x)], 
#         [min(x)*model.coef_[0][0]+model.intercept_, max(x)*model.coef_[0][0]+model.intercept_],
#         color='red')
# plt.show()

In [110]:
temp = pd.DataFrame({'g': np.concatenate([df['h g'], df['a g']]),
                     'elo diff': np.concatenate([df['h elo']-df['a elo'],
                                                 df['a elo']-df['h elo']]),
                     'g form diff': np.concatenate([df['h g form']-df['a g form'],
                                                    df['a g form']-df['h g form']]),
                     'npxg form diff': np.concatenate([df['h npxg form']-df['a npxg form'],
                                                       df['a npxg form']-df['h npxg form']])})
# Drop rows with nans
drops = np.unique(np.where(temp.isna())[0])
temp = temp.drop(index=drops).reset_index(drop=True)
temp
# Create x and y
x = temp.iloc[:,1:]
y = temp['g'].values
# Create classes in y. 0 = CS, 1 = goal(s) conceded
y = np.array([0 if y[i]==0 else 1 for i in range(len(y))])


# """ Viz of classes on 3-D data """
# %matplotlib notebook
# fig = plt.figure(figsize=[10,8])
# ax = fig.add_subplot(projection='3d')
# ax.scatter(x.iloc[:,0], x.iloc[:,1], x.iloc[:,2], c=y, cmap='jet', alpha=0.2)
# ax.set_xlabel('ELO diff')
# ax.set_ylabel('G form diff')
# ax.set_zlabel('npxG form diff')
# fig.tight_layout()
# plt.show()


# """ PCA """
# U, s, VT = np.linalg.svd(x, full_matrices=False)
# x = x.values @ VT[:,:2]
# %matplotlib inline
# fig = plt.figure(figsize=[12,8])
# ax = fig.add_subplot()
# ax.scatter(x[:,0], x[:,1], c=y, cmap='jet', alpha=0.2)
# plt.show()


""" Fit logistic regression model """
model = LogisticRegression().fit(x,y)
y_hat = np.argmax(model.predict_proba(x), axis=1)
tp = len(np.where(np.logical_and(y_hat==0, y==0))[0])
fp = len(np.where(np.logical_and(y_hat==0, y==1))[0])
tn = len(np.where(np.logical_and(y_hat==1, y==1))[0])
fn = len(np.where(np.logical_and(y_hat==1, y==0))[0])
chart = pd.DataFrame(columns=['CS', 'Not CS'], index=['Predicted CS', 'Predicted Not CS'], data=[[tp, fp], [fn, tn]])
print(chart)

        
# # model = KMeans(n_clusters=2).fit(x, y)
# # %matplotlib notebook
# # fig = plt.figure(figsize=[10,8])
# # ax = fig.add_subplot(projection='3d')
# # ax.scatter(x.iloc[:,0], x.iloc[:,1], x.iloc[:,2], c=model.labels_, cmap='jet', alpha=0.2)
# # ax.set_xlabel('ELO diff')
# # ax.set_ylabel('G form diff')
# # ax.set_zlabel('npxG form diff')
# # fig.tight_layout()
# # plt.show()

                   CS  Not CS
Predicted CS      101      96
Predicted Not CS  788    2017


#### Using 5 matches to determine form:

|                  |   CS | Not CS
|             ---- | ---- |   ----
|     Predicted CS |  102 |     94
| Predicted Not CS |  787 |   2019

#### Using 10 matches to determine form:

|                  |   CS | Not CS
|             ---- | ---- |   ----
|     Predicted CS |  112 |     97
| Predicted Not CS |  777 |   2016

#### Using 15 matches to determine form:

|                  |   CS | Not CS
|             ---- | ---- |   ----
|     Predicted CS |  105 |     93
| Predicted Not CS |  784 |   2020

#### Using 20 matches to determine form:

|                  |   CS | Not CS
|             ---- | ---- |   ----
|     Predicted CS |  104 |     96 
| Predicted Not CS |  785 |   2017